In [1]:
import scipy.sparse

In [2]:
# Load the channel tuple sparse matrix
S = scipy.sparse.load_npz('/dlabdata1/youtube_large/jouven/final_sparse_matrix/sparse_matrix_for_word2vec.npz')

In [3]:
S.count_nonzero()

2834601008

In [6]:
S.shape

(57004, 406925230)

In [11]:
'''
Create a text file named word_vocabulary, the word here corresponds to each channel
Each line corresponds to a (channel, # occurences) pair and written as
"channel_index #occurences" in the file
'''
def create_word_vocabulary(S):
    word_pair = ""
    occurences_tab = S.sum(axis = 1).flatten().tolist()[0]  # sum the rows
    for idx, occurences in enumerate(occurences_tab):
        #idx represents the channel index
        word_pair += str(idx) + " " + str(occurences) + "\n"
    print('Writing into word_vocabulary...')
    f = open("/dlabdata1/youtube_large/jouven/word2vecf_preprocessing/word_vocabulary", "w")
    f.write(word_pair)
    f.close()
        
    

In [3]:
'''
Create a text file named context_vocabulary, the context here is the uset
Each line corresponds to a (user, # occurences) pair and written as
"user_index #occurences" in the file
'''
def create_context_vocabulary(S):
    context_pair = ""
    # Temporary variable for speed purpose
    context_pair_temp = ""
    occurences_tab = S.sum(axis = 0).flatten().tolist()[0]  # sum the columns
    for idx, occurences in enumerate(occurences_tab):
        if idx % 10000 == 0 and idx > 0:
            context_pair += context_pair_temp
            context_pair_temp = ""
            context_pair_temp += str(idx) + " " + str(occurences) + "\n"
    print('Writing into context_vocabulary...')
    f = open("/dlabdata1/youtube_large/jouven/word2vecf_preprocessing/context_vocabulary", "w")
    f.write(context_pair)
    f.close()

In [12]:
create_word_vocabulary(S)

Writing into word_vocabulary...


In [4]:
create_context_vocabulary(S)

Writing into context_vocabulary...
